### This notebook dedups localities within the same year

#### Multipe string distance metrics are used. Booth number also used

Required libraries: `pandas`, `recordlinkage`, `numpy`

In [1]:
import pandas as pd
import recordlinkage
import recordlinkage.preprocessing
import numpy as np

In [2]:
new_df = pd.read_excel('new_booth_to_local_map.xlsx')

For each locality, find the average booth number. This is useful when trying to dedup localites. Duplicate localities shouldn't be very far off from each other.

In [3]:
new_localities = new_df.groupby('name2')['booth_num'].mean().reset_index()

Get the metaphone (the way the locality 'sounds'). Not using this currently because not reliable.

In [4]:
new_localities['metaphone'] = recordlinkage.preprocessing.phonetic(new_df.name2,method="metaphone")

Boilerplate code for creating 'match pairs'. Each locality is matched with every other locality, so this creates `multi-index` pairs.

In [5]:
indexer = recordlinkage.Index()
indexer.full()
candidate_links = indexer.index(new_localities)

In [16]:
candidate_links

MultiIndex([(  1,   0),
            (  2,   0),
            (  2,   1),
            (  3,   0),
            (  3,   1),
            (  3,   2),
            (  4,   0),
            (  4,   1),
            (  4,   2),
            (  4,   3),
            ...
            (192, 182),
            (192, 183),
            (192, 184),
            (192, 185),
            (192, 186),
            (192, 187),
            (192, 188),
            (192, 189),
            (192, 190),
            (192, 191)],
           length=18528)

This is where the comparision happens:

In [6]:
compare_booths = recordlinkage.Compare()

#Get Levenshtein score booth name
compare_booths.string('name2', 'name2', method='levenshtein', threshold=None, label='name_lv_score')

#Get Jaro-Winkler Score for booth name
compare_booths.string('name2', 'name2', method='jarowinkler', threshold=None, label='name_jw_score')

#Get Jaro-winkler score for the way the name is pronounced
compare_booths.string('metaphone', 'metaphone', method='levenshtein', threshold=None, label='metaphone')

#Get score for how far apart the booth numbers are
compare_booths.numeric('booth_num', 'booth_num', label='booth_number_score',method="gauss",offset=3,scale=5)

<Compare>

Run the comparision:

In [7]:
features = compare_booths.compute(candidate_links, new_localities)

Consolidate scores in a single frame

In [8]:
#match_index = features[features.sum(axis=1)>0].index.values
#match_index = cl.fit_predict(features).values
match_index = features.index.values
master_name_index = [x[0] for x in match_index]
master_names = new_localities.iloc[master_name_index]

duplicate_name_index = [x[1] for x in match_index]
duplicate_names = new_localities.iloc[duplicate_name_index]

master_names.rename(columns={"name2":'master_name',"booth_num":"new_booth_num"},inplace=True)

/mnt/c/Users/gufra/OneDrive/Documents/Workspace/Jutaav/code_base/aurangabad/venv/lib/python3.6/site-packages/pandas/core/frame.py:4301: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [9]:
master_names['duplicate_name'] = duplicate_names.name2.tolist()
master_names['duplicate_booth_num'] = duplicate_names.booth_num.tolist()
master_names['name_jw_score'] = features.name_jw_score.tolist()
master_names['name_lv_score'] = features.name_lv_score.tolist()
master_names['booth_score'] = features.booth_number_score.tolist()
master_names['metaphone'] = features.metaphone.tolist()
#master_names['duplicate_candidate_name'] = duplicate_names.name2.tolist()

/mnt/c/Users/gufra/OneDrive/Documents/Workspace/Jutaav/code_base/aurangabad/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/mnt/c/Users/gufra/OneDrive/Documents/Workspace/Jutaav/code_base/aurangabad/venv/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/mnt/c/Users/gufra/OneDrive/Documents/Workspace/Jutaav/code_base/aurangabad/venv/lib/python3.6/site-pac

In [11]:
master_names['average_score'] = master_names[['name_jw_score','name_lv_score','booth_score']]\
                                .apply(lambda x:x.mean(),axis=1).round(2)

/mnt/c/Users/gufra/OneDrive/Documents/Workspace/Jutaav/code_base/aurangabad/venv/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [12]:
master_names = master_names[['master_name','duplicate_name','new_booth_num','duplicate_booth_num',
                            'name_jw_score',"name_lv_score",'booth_score','average_score']]

In [13]:
master_names.round(5).to_csv('data/dedup_new.csv',index=False)


In [17]:
dups_df = master_names[master_names.average_score>=0.8].sort_values(by=['average_score'],ascending=False)
dups_df

,master_name,duplicate_name,new_booth_num,duplicate_booth_num,name_jw_score,name_lv_score,booth_score,average_score
34,BHARTHAULI,BHARATHAULI,19.0,22.0,0.961818,0.909091,1.0,0.96
44,CHANDPUR,CHANDAPUR,291.0,290.0,0.977778,0.888889,1.0,0.96
158,POKHRAHAN,POKHRAHA,64.0,65.0,0.977778,0.888889,1.0,0.96
36,BHATKUR,BHATAKUR,319.0,318.0,0.975000,0.875000,1.0,0.95
134,NAUGADH,NAUGAD,94.0,93.0,0.971429,0.857143,1.0,0.94
67,DOSMHA,DOSMA,321.0,322.0,0.966667,0.833333,1.0,0.93
69,DUMRI,DUMARI,315.0,316.0,0.961111,0.833333,1.0,0.93
148,PARSA,PARASA,99.0,100.0,0.914444,0.833333,1.0,0.92
124,MANDARAPURA,MADARPURA,75.0,77.0,0.892256,0.818182,1.0,0.90
151,PAWAI,PAVAI,222.0,222.0,0.893333,0.800000,1.0,0.90


In [18]:
dups_df.to_csv('data/dupicate_localities.csv',index=False)